In [ ]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns

In [ ]:
df_ = pd.read_csv("https://raw.githubusercontent.com/thistleknot/Python-Stock/master/data/raw/states.csv").set_index('States')

In [ ]:
df = sm.add_constant(df_)

In [ ]:
'''
a = (56, 128.6, 12, 123.8, 64.34, 78, 763.3)
b = (1.1, 2.9, 4.2)
#Use the trim keyword to perform a trimmed (Yuen) t-test. For example, using 20% trimming, trim=.2, the test will reduce the impact of one (np.floor(trim*len(a))) element from each tail of sample a. It will have no effect on sample b because np.floor(trim*len(b)) is 0.

scipy.stats.ttest_ind(a, b, trim=.2).pvalue
#Ttest_indResult(statistic=3.4463884028073513,
                #pvalue=0.01369338726499547)
'''

In [ ]:
def f_test(x, y, alt="two_sided"):
    """
    Calculates the F-test.
    :param x: The first group of data
    :param y: The second group of data
    :param alt: The alternative hypothesis, one of "two_sided" (default), "greater" or "less"
    :return: a tuple with the F statistic value and the p-value.
    """
    df1 = len(x) - 1
    df2 = len(y) - 1
    f = x.var() / y.var()
    if alt == "greater":
        p = 1.0 - st.f.cdf(f, df1, df2)
    elif alt == "less":
        p = st.f.cdf(f, df1, df2)
    else:
        # two-sided by default
        # Crawley, the R book, p.355
        p = 2.0*(1.0 - st.f.cdf(f, df1, df2))
    return f, p

In [17]:

#nested functions (kind of like classes, get access to parent functions members).
def filter_df(x,y):
    
    def print_df(aboveCenter,belowCenter,center):
        
        data = df[[y,*x]].sort_values(kind="quicksort", by=y, ascending=True)
        #abovePos = [i for i, item in enumerate(data.columns) if item in aboveMedian]
        #belowPos = [i for i, item in enumerate(data.columns) if item in belowMedian]

        print("all group")
        print(data.describe())
        med = pd.DataFrame(np.median(data,axis=0)).T
        means = pd.DataFrame(np.mean(data,axis=0)).T
        
        med.columns = data.columns
        
        filtered = data.copy()
        
        if(center):
            centers = means
            
        else:
            centers = med

        filtered = filtered.iloc[pd.DataFrame(np.array(filtered[list(aboveCenter)])>np.array(centers[list(aboveCenter)])).replace(False,np.NaN).dropna().index]

        filtered = filtered.iloc[pd.DataFrame(np.array(filtered[list(belowCenter)])<np.array(centers[list(belowCenter)])).replace(False,np.NaN).dropna().index]

        #opposite = ([i for i, item in enumerate(data.columns) if item not in [filtered.index]])
        oppositefilter = data.copy()

        oppositefilter = oppositefilter.iloc[pd.DataFrame(np.array(oppositefilter[list(aboveCenter)])<np.array(centers[list(aboveCenter)])).replace(False,np.NaN).dropna().index]     
        
        oppositefilter = oppositefilter.iloc[pd.DataFrame(np.array(oppositefilter[list(belowCenter)])>np.array(centers[list(belowCenter)])).replace(False,np.NaN).dropna().index]    

        together = np.concatenate([filtered.index,oppositefilter.index])
        
        neither = [i for i, item in enumerate(list(df.index)) if item not in together]
        
        neitherfilter = data.copy()
        neitherfilter = neitherfilter.iloc[neither]
        
        '''
        A=notfiltered
        B=filtered
        H0: A>=B
        H1: B>A
        '''
        
        quantiles_7num = [.02,.09,.25,.5,.75,.91,.98]
        
        q_s_a = data[y].quantile(quantiles_7num)
        q_s_i = filtered[y].quantile(quantiles_7num)
        q_s_o = oppositefilter[y].quantile(quantiles_7num)
        q_s_n = neitherfilter[y].quantile(quantiles_7num)
                
        print("all mapped with in and out group")
        plt.plot(data[y])
        plt.plot(filtered[y])
        plt.plot(oppositefilter[y])
        plt.xticks(rotation = 45,size=8)
        plt.show()
        plt.plot(q_s_a)
        plt.plot(q_s_i)
        plt.plot(q_s_o)
        plt.show()
        print("all mapped against neither group")
        plt.plot(data[y])
        plt.plot(neitherfilter[y])
        plt.xticks(rotation = 45,size=8)
        plt.show()
        plt.plot(q_s_a)
        plt.plot(q_s_n)
        plt.show()
        
        data1 = pd.DataFrame(data[[y]]).assign(Location=1)[['Location',y]]
        data2 = pd.DataFrame(filtered[[y]]).assign(Location=2)[['Location',y]]
        data3 = pd.DataFrame(oppositefilter[[y]]).assign(Location=3)[['Location',y]]
        data4 = pd.DataFrame(neitherfilter[[y]]).assign(Location=4)[['Location',y]]
        
        #print(data4)
        
        data5 = pd.concat([data2,data3,data4],axis=0)[['Location']]
        
        data.loc[data5.index]['cluster']=data5['Location']
        #print(data5)
        #data['group']=data5
        
        cdf = pd.concat([data1, data2, data3, data4])    
        mdf = pd.melt(cdf, id_vars=['Location'], var_name=['Group'])
        #print(mdf.head())
        
        ax = sns.boxplot(x="Location", y="value", hue="Group", data=mdf)    
        plt.show()
        '''
        #fig = mpf.figure(figsize=(10, 7), style=s)
        fig = plt.figure(figsize=plt.figaspect(2.))
        fig.suptitle('boxplots')
        #fig = Figure()
        #FigureCanvas(fig) # not needed in mpl >= 3.1
        ax1 = fig.add_subplot(6,2,1)
        ax2 = fig.add_subplot(6,2,2)
        ax3 = fig.add_subplot(6,2,3)
        ax4 = fig.add_subplot(6,2,4)
        ax5 = fig.add_subplot(6,2,5)
        ax6 = fig.add_subplot(6,2,6)

        ax1.boxplot(data[y])
        ax2.boxplot(filtered[y])
        ax3.boxplot(data[y])
        ax4.boxplot(oppositefilter[y])
        ax5.boxplot(data[y])
        ax6.boxplot(neitherfilter[y])
        plt.show()
        '''
        print("all mapped with in and out group")
        data[y].hist()
        filtered[y].hist()
        oppositefilter[y].hist()
        plt.show()
        print("all mapped against neither group")
        data[y].hist()
        neitherfilter[y].hist()
        plt.show()
        
        print("in group")
        sp = (st.ttest_ind(filtered[y],data[y],equal_var=False))
        print(sp.pvalue/2)
        #print(st.ttest_1samp(a=filtered[y],popmean=np.mean(data[y])))
        print("f/variance test:",f_test(filtered[y],data[y]))
        #if(sp.pvalue/2<.05):
            #print("reject null hypothesis, B(filter)>A")
        #df_ = len(filtered[y]) + len(data[y]) - 2
        display(filtered.describe())
        display(filtered)
        
        print("out group")
        sp = (st.ttest_ind(oppositefilter[y],data[y],equal_var=False))
        print(sp.pvalue/2)
        #print(st.ttest_1samp(a=oppositefilter[y],popmean=np.mean(data[y])))
        print("f/variance test:",f_test(oppositefilter[y],data[y]))
        #if(sp.pvalue/2<.05):
            #print("reject null hypothesis, B(filter)>A")
        #df_ = len(filtered[y]) + len(data[y]) - 2
        display(oppositefilter.describe())
        display(oppositefilter)
        
        #return([filtered,oppositefilter])
        print("neither group")
        sp = (st.ttest_ind(neitherfilter[y],data[y],equal_var=False))
        print(sp.pvalue/2)
        #print(st.ttest_1samp(a=neitherfilter[y],popmean=np.mean(data[y])))
        print("f/variance test:",f_test(neitherfilter[y],data[y]))
        display(neitherfilter.describe())
        display(neitherfilter)
        
    aboveCenter = widgets.SelectMultiple(
        options=df[list(x)].columns,
        description='AboveMedian',
        disabled=False
    )
    
    belowCenter = widgets.SelectMultiple(
        options=df[list(x)].columns,
        description='BelowMedian',
        disabled=False
    )
    
    center=widgets.Checkbox(
        description='Center: [Checked: Mean, Unchecked: Median]',
        value=False,
        disabled=False,
        indent=False)

    interact(print_df,aboveCenter=aboveCenter,belowCenter=belowCenter,center=center)
    
def derive_xnames(y):
    x_ = widgets.SelectMultiple(
        options=df.columns[~df.columns.isin([y])],
        value=list(df.columns[~df.columns.isin([y])]),
        disabled=False
    )
    
    interact(filter_df,x=x_,y=y)

In [18]:

y=widgets.Select(options=df_.columns,disabled=False)

interact(derive_xnames,y=y)


interactive(children=(Select(description='y', options=('Poverty', 'Infant Mort', 'White', 'Crime', 'Doctors', …

<function __main__.derive_xnames(y)>